# Transformation from BILUO, IOB to JSONL 

In this notebook there are some procedure to transform IOB scheme to BILUO scheme, and then to JSONL format for Spacy.

## Import libraries

In [1]:
import pandas as pd
import spacy
from spacy.gold import offsets_from_biluo_tags, spans_from_biluo_tags

## Read data

In [2]:
data = pd.read_csv("../data/ner_dataset.csv", encoding='latin1')
data = data.fillna(method='ffill')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [3]:
class SentenceGetter(object):
    """Class to Get the sentence in this format:
    [(Token_1, Part_of_Speech_1, Tag_1), ..., (Token_n, Part_of_Speech_1, Tag_1)]"""
    def __init__(self, data):
        """Args:
            data is the pandas.DataFrame which contains the above dataset"""
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        """Return one sentence"""
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None
    
    def get_sentences(self):
        return self.sentences

In [4]:
getter = SentenceGetter(data)
sentences = getter.get_sentences()

## Transform IOB schema to BILUO

In [5]:
biluo_dataset = []
text = []
tags = []
for n, sentence in enumerate(sentences):
    for i, element in enumerate(sentence):
        new_tag = ''
        tag = element[1]
        try:
            if tag.startswith('B'):
                if sentence[i+1][1].startswith('O'):
                    new_tag = tag.replace("B", "U")
                else:
                    new_tag = tag
            if tag.startswith('I'):
                if sentence[i+1][1].startswith('O'):
                    new_tag = tag.replace("I", "L")
                else:
                    new_tag = tag
            if tag.startswith('O'):
                new_tag = tag
        except IndexError:
            continue
        biluo_dataset.append(("Sentence:" + str(n), element[0], new_tag))

In [6]:
biluo_dataset

[('Sentence:0', 'Thousands', 'O'),
 ('Sentence:0', 'of', 'O'),
 ('Sentence:0', 'demonstrators', 'O'),
 ('Sentence:0', 'have', 'O'),
 ('Sentence:0', 'marched', 'O'),
 ('Sentence:0', 'through', 'O'),
 ('Sentence:0', 'London', 'U-geo'),
 ('Sentence:0', 'to', 'O'),
 ('Sentence:0', 'protest', 'O'),
 ('Sentence:0', 'the', 'O'),
 ('Sentence:0', 'war', 'O'),
 ('Sentence:0', 'in', 'O'),
 ('Sentence:0', 'Iraq', 'U-geo'),
 ('Sentence:0', 'and', 'O'),
 ('Sentence:0', 'demand', 'O'),
 ('Sentence:0', 'the', 'O'),
 ('Sentence:0', 'withdrawal', 'O'),
 ('Sentence:0', 'of', 'O'),
 ('Sentence:0', 'British', 'U-gpe'),
 ('Sentence:0', 'troops', 'O'),
 ('Sentence:0', 'from', 'O'),
 ('Sentence:0', 'that', 'O'),
 ('Sentence:0', 'country', 'O'),
 ('Sentence:0', '.', 'O'),
 ('Sentence:1', 'Iranian', 'U-gpe'),
 ('Sentence:1', 'officials', 'O'),
 ('Sentence:1', 'say', 'O'),
 ('Sentence:1', 'they', 'O'),
 ('Sentence:1', 'expect', 'O'),
 ('Sentence:1', 'to', 'O'),
 ('Sentence:1', 'get', 'O'),
 ('Sentence:1', 'acces

In [7]:
df_biluo = pd.DataFrame(biluo_dataset, columns=["Sentence #", "Word", "Tag"] )

In [8]:
df_biluo.head()

,Sentence #,Word,Tag
0,Sentence:0,Thousands,O
1,Sentence:0,of,O
2,Sentence:0,demonstrators,O
3,Sentence:0,have,O
4,Sentence:0,marched,O


In [9]:
df_biluo.to_csv("../data/biluo_ner_dataset.csv", index=False)

## Transform BILUO scheme to JSONL format

In [10]:
!python -m spacy download en_core_web_sm

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [11]:
getter = SentenceGetter(df_biluo)
sentences = getter.get_sentences()

In [12]:
from spacy.gold import offsets_from_biluo_tags, spans_from_biluo_tags

nlp = spacy.load('en_core_web_sm')

dataset = []

for n, sentence in enumerate(sentences):
    text = ""
    tags = []
    for element in sentence:
        text = text + element[0] + " "
        tags.append(element[1])
    doc = nlp(text.strip())
    entities = offsets_from_biluo_tags(doc, tags)
    print(str(n) + "/" + str(len(sentences)))
    dataset.append((text, {"entities": entities}))

0/47959
1/47959
2/47959
3/47959
4/47959
5/47959
6/47959
7/47959
8/47959
9/47959
10/47959
11/47959
12/47959
13/47959
14/47959
15/47959
16/47959
17/47959
18/47959
19/47959
20/47959
21/47959
22/47959
23/47959
24/47959
25/47959
26/47959
27/47959
28/47959
29/47959
30/47959
31/47959
32/47959
33/47959
34/47959
35/47959
36/47959
37/47959
38/47959
39/47959
40/47959
41/47959
42/47959
43/47959
44/47959
45/47959
46/47959
47/47959
48/47959
49/47959
50/47959
51/47959
52/47959
53/47959
54/47959
55/47959
56/47959
57/47959
58/47959
59/47959
60/47959
61/47959
62/47959
63/47959
64/47959
65/47959
66/47959
67/47959
68/47959
69/47959
70/47959
71/47959
72/47959
73/47959
74/47959
75/47959
76/47959
77/47959
78/47959
79/47959
80/47959
81/47959
82/47959
83/47959
84/47959
85/47959
86/47959
87/47959
88/47959
89/47959
90/47959
91/47959
92/47959
93/47959
94/47959
95/47959
96/47959
97/47959
98/47959
99/47959
100/47959
101/47959
102/47959
103/47959
104/47959
105/47959
106/47959
107/47959
108/47959
109/47959
110/47959


832/47959
833/47959
834/47959
835/47959
836/47959
837/47959
838/47959
839/47959
840/47959
841/47959
842/47959
843/47959
844/47959
845/47959
846/47959
847/47959
848/47959
849/47959
850/47959
851/47959
852/47959
853/47959
854/47959
855/47959
856/47959
857/47959
858/47959
859/47959
860/47959
861/47959
862/47959
863/47959
864/47959
865/47959
866/47959
867/47959
868/47959
869/47959
870/47959
871/47959
872/47959
873/47959
874/47959
875/47959
876/47959
877/47959
878/47959
879/47959
880/47959
881/47959
882/47959
883/47959
884/47959
885/47959
886/47959
887/47959
888/47959
889/47959
890/47959
891/47959
892/47959
893/47959
894/47959
895/47959
896/47959
897/47959
898/47959
899/47959
900/47959
901/47959
902/47959
903/47959
904/47959
905/47959
906/47959
907/47959
908/47959
909/47959
910/47959
911/47959
912/47959
913/47959
914/47959
915/47959
916/47959
917/47959
918/47959
919/47959
920/47959
921/47959
922/47959
923/47959
924/47959
925/47959
926/47959
927/47959
928/47959
929/47959
930/47959
931/47959


1602/47959
1603/47959
1604/47959
1605/47959
1606/47959
1607/47959
1608/47959
1609/47959
1610/47959
1611/47959
1612/47959
1613/47959
1614/47959
1615/47959
1616/47959
1617/47959
1618/47959
1619/47959
1620/47959
1621/47959
1622/47959
1623/47959
1624/47959
1625/47959
1626/47959
1627/47959
1628/47959
1629/47959
1630/47959
1631/47959
1632/47959
1633/47959
1634/47959
1635/47959
1636/47959
1637/47959
1638/47959
1639/47959
1640/47959
1641/47959
1642/47959
1643/47959
1644/47959
1645/47959
1646/47959
1647/47959
1648/47959
1649/47959
1650/47959
1651/47959
1652/47959
1653/47959
1654/47959
1655/47959
1656/47959
1657/47959
1658/47959
1659/47959
1660/47959
1661/47959
1662/47959
1663/47959
1664/47959
1665/47959
1666/47959
1667/47959
1668/47959
1669/47959
1670/47959
1671/47959
1672/47959
1673/47959
1674/47959
1675/47959
1676/47959
1677/47959
1678/47959
1679/47959
1680/47959
1681/47959
1682/47959
1683/47959
1684/47959
1685/47959
1686/47959
1687/47959
1688/47959
1689/47959
1690/47959
1691/47959
1692/47959

2351/47959
2352/47959
2353/47959
2354/47959
2355/47959
2356/47959
2357/47959
2358/47959
2359/47959
2360/47959
2361/47959
2362/47959
2363/47959
2364/47959
2365/47959
2366/47959
2367/47959
2368/47959
2369/47959
2370/47959
2371/47959
2372/47959
2373/47959
2374/47959
2375/47959
2376/47959
2377/47959
2378/47959
2379/47959
2380/47959
2381/47959
2382/47959
2383/47959
2384/47959
2385/47959
2386/47959
2387/47959
2388/47959
2389/47959
2390/47959
2391/47959
2392/47959
2393/47959
2394/47959
2395/47959
2396/47959
2397/47959
2398/47959
2399/47959
2400/47959
2401/47959
2402/47959
2403/47959
2404/47959
2405/47959
2406/47959
2407/47959
2408/47959
2409/47959
2410/47959
2411/47959
2412/47959
2413/47959
2414/47959
2415/47959
2416/47959
2417/47959
2418/47959
2419/47959
2420/47959
2421/47959
2422/47959
2423/47959
2424/47959
2425/47959
2426/47959
2427/47959
2428/47959
2429/47959
2430/47959
2431/47959
2432/47959
2433/47959
2434/47959
2435/47959
2436/47959
2437/47959
2438/47959
2439/47959
2440/47959
2441/47959

3104/47959
3105/47959
3106/47959
3107/47959
3108/47959
3109/47959
3110/47959
3111/47959
3112/47959
3113/47959
3114/47959
3115/47959
3116/47959
3117/47959
3118/47959
3119/47959
3120/47959
3121/47959
3122/47959
3123/47959
3124/47959
3125/47959
3126/47959
3127/47959
3128/47959
3129/47959
3130/47959
3131/47959
3132/47959
3133/47959
3134/47959
3135/47959
3136/47959
3137/47959
3138/47959
3139/47959
3140/47959
3141/47959
3142/47959
3143/47959
3144/47959
3145/47959
3146/47959
3147/47959
3148/47959
3149/47959
3150/47959
3151/47959
3152/47959
3153/47959
3154/47959
3155/47959
3156/47959
3157/47959
3158/47959
3159/47959
3160/47959
3161/47959
3162/47959
3163/47959
3164/47959
3165/47959
3166/47959
3167/47959
3168/47959
3169/47959
3170/47959
3171/47959
3172/47959
3173/47959
3174/47959
3175/47959
3176/47959
3177/47959
3178/47959
3179/47959
3180/47959
3181/47959
3182/47959
3183/47959
3184/47959
3185/47959
3186/47959
3187/47959
3188/47959
3189/47959
3190/47959
3191/47959
3192/47959
3193/47959
3194/47959

...

45003/47959
45004/47959
45005/47959
45006/47959
45007/47959
45008/47959
45009/47959
45010/47959
45011/47959
45012/47959
45013/47959
45014/47959
45015/47959
45016/47959
45017/47959
45018/47959
45019/47959
45020/47959
45021/47959
45022/47959
45023/47959
45024/47959
45025/47959
45026/47959
45027/47959
45028/47959
45029/47959
45030/47959
45031/47959
45032/47959
45033/47959
45034/47959
45035/47959
45036/47959
45037/47959
45038/47959
45039/47959
45040/47959
45041/47959
45042/47959
45043/47959
45044/47959
45045/47959
45046/47959
45047/47959
45048/47959
45049/47959
45050/47959
45051/47959
45052/47959
45053/47959
45054/47959
45055/47959
45056/47959
45057/47959
45058/47959
45059/47959
45060/47959
45061/47959
45062/47959
45063/47959
45064/47959
45065/47959
45066/47959
45067/47959
45068/47959
45069/47959
45070/47959
45071/47959
45072/47959
45073/47959
45074/47959
45075/47959
45076/47959
45077/47959
45078/47959
45079/47959
45080/47959
45081/47959
45082/47959
45083/47959
45084/47959
45085/47959
4508

45687/47959
45688/47959
45689/47959
45690/47959
45691/47959
45692/47959
45693/47959
45694/47959
45695/47959
45696/47959
45697/47959
45698/47959
45699/47959
45700/47959
45701/47959
45702/47959
45703/47959
45704/47959
45705/47959
45706/47959
45707/47959
45708/47959
45709/47959
45710/47959
45711/47959
45712/47959
45713/47959
45714/47959
45715/47959
45716/47959
45717/47959
45718/47959
45719/47959
45720/47959
45721/47959
45722/47959
45723/47959
45724/47959
45725/47959
45726/47959
45727/47959
45728/47959
45729/47959
45730/47959
45731/47959
45732/47959
45733/47959
45734/47959
45735/47959
45736/47959
45737/47959
45738/47959
45739/47959
45740/47959
45741/47959
45742/47959
45743/47959
45744/47959
45745/47959
45746/47959
45747/47959
45748/47959
45749/47959
45750/47959
45751/47959
45752/47959
45753/47959
45754/47959
45755/47959
45756/47959
45757/47959
45758/47959
45759/47959
45760/47959
45761/47959
45762/47959
45763/47959
45764/47959
45765/47959
45766/47959
45767/47959
45768/47959
45769/47959
4577

46377/47959
46378/47959
46379/47959
46380/47959
46381/47959
46382/47959
46383/47959
46384/47959
46385/47959
46386/47959
46387/47959
46388/47959
46389/47959
46390/47959
46391/47959
46392/47959
46393/47959
46394/47959
46395/47959
46396/47959
46397/47959
46398/47959
46399/47959
46400/47959
46401/47959
46402/47959
46403/47959
46404/47959
46405/47959
46406/47959
46407/47959
46408/47959
46409/47959
46410/47959
46411/47959
46412/47959
46413/47959
46414/47959
46415/47959
46416/47959
46417/47959
46418/47959
46419/47959
46420/47959
46421/47959
46422/47959
46423/47959
46424/47959
46425/47959
46426/47959
46427/47959
46428/47959
46429/47959
46430/47959
46431/47959
46432/47959
46433/47959
46434/47959
46435/47959
46436/47959
46437/47959
46438/47959
46439/47959
46440/47959
46441/47959
46442/47959
46443/47959
46444/47959
46445/47959
46446/47959
46447/47959
46448/47959
46449/47959
46450/47959
46451/47959
46452/47959
46453/47959
46454/47959
46455/47959
46456/47959
46457/47959
46458/47959
46459/47959
4646

47065/47959
47066/47959
47067/47959
47068/47959
47069/47959
47070/47959
47071/47959
47072/47959
47073/47959
47074/47959
47075/47959
47076/47959
47077/47959
47078/47959
47079/47959
47080/47959
47081/47959
47082/47959
47083/47959
47084/47959
47085/47959
47086/47959
47087/47959
47088/47959
47089/47959
47090/47959
47091/47959
47092/47959
47093/47959
47094/47959
47095/47959
47096/47959
47097/47959
47098/47959
47099/47959
47100/47959
47101/47959
47102/47959
47103/47959
47104/47959
47105/47959
47106/47959
47107/47959
47108/47959
47109/47959
47110/47959
47111/47959
47112/47959
47113/47959
47114/47959
47115/47959
47116/47959
47117/47959
47118/47959
47119/47959
47120/47959
47121/47959
47122/47959
47123/47959
47124/47959
47125/47959
47126/47959
47127/47959
47128/47959
47129/47959
47130/47959
47131/47959
47132/47959
47133/47959
47134/47959
47135/47959
47136/47959
47137/47959
47138/47959
47139/47959
47140/47959
47141/47959
47142/47959
47143/47959
47144/47959
47145/47959
47146/47959
47147/47959
4714

47748/47959
47749/47959
47750/47959
47751/47959
47752/47959
47753/47959
47754/47959
47755/47959
47756/47959
47757/47959
47758/47959
47759/47959
47760/47959
47761/47959
47762/47959
47763/47959
47764/47959
47765/47959
47766/47959
47767/47959
47768/47959
47769/47959
47770/47959
47771/47959
47772/47959
47773/47959
47774/47959
47775/47959
47776/47959
47777/47959
47778/47959
47779/47959
47780/47959
47781/47959
47782/47959
47783/47959
47784/47959
47785/47959
47786/47959
47787/47959
47788/47959
47789/47959
47790/47959
47791/47959
47792/47959
47793/47959
47794/47959
47795/47959
47796/47959
47797/47959
47798/47959
47799/47959
47800/47959
47801/47959
47802/47959
47803/47959
47804/47959
47805/47959
47806/47959
47807/47959
47808/47959
47809/47959
47810/47959
47811/47959
47812/47959
47813/47959
47814/47959
47815/47959
47816/47959
47817/47959
47818/47959
47819/47959
47820/47959
47821/47959
47822/47959
47823/47959
47824/47959
47825/47959
47826/47959
47827/47959
47828/47959
47829/47959
47830/47959
4783

In [14]:
import pickle

with open('ner_jsonl_dataset.pickle', 'wb') as fp:
    pickle.dump(dataset, fp)